# users table에 패스워드 넣기

In [1]:
!pip install pymysql > /dev/null

������ ��θ� ã�� �� �����ϴ�.


In [2]:
import json
with open('mysql.json') as fp:
    config_str = fp.read()
config = json.loads(config_str)

In [11]:
import pymysql
conn = pymysql.connect(
    host=config['host'],
    user=config['user'],
    password=config['password'],
    database=config['database'],
    port=config['port']

)

## 패스워드 암호화
- Secure Hash Algorithm(SHA-256)

In [3]:
import hashlib

pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

- Base64로 인코딩

In [4]:
import base64

base64.b64encode(pwd_sha256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [5]:
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [6]:
len(hashed_pwd)

44

- 패스워드를 생성하는 함수

In [8]:
import hashlib, base64
def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
    return hashed_pwd

In [9]:
gen_pwd('qwerty')

'ZehL4zUy+3hMSBKWdfnv86aCsnFowOp0Syz1juAjN8U='

# users table에 패스워드 넣기
- uid + 21

In [12]:
sql = 'select uid from users;'
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',),
 ('djy',),
 ('eskim',),
 ('gdhong',),
 ('gdhong2',),
 ('jbpark',),
 ('jbpark2',),
 ('wjlee',))

In [13]:
sql_update = 'update users set pwd=%s where uid=%s;'
for line in results:
    uid = line[0]
    pwd = gen_pwd(uid+'21')
    cur.execute(sql_update, (pwd, uid))
    conn.commit()

- email 추가: uid@mc.com

In [14]:
sql_email = 'update users set email=%s where uid=%s;'
for line in results:
    uid = line[0]
    email = uid + '%mc.com'
    cur.execute(sql_email, (email, uid))
conn.commit()

## 로그인

In [15]:
sql_login = 'select uid, pwd from users where uid=%s and is_deleted=0'

- Case 1) uid가 없는 경우

In [16]:
# result가 Null
uid = 'fool'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result

- Case 2) 비밀번호가 틀린 경우

In [20]:
uid = 'djy'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result

('djy', 'mxsnWYLpB/nAOt37fpt3PMXpGGQprktT3vm2p9RfFyY=')

In [21]:
pwd = 'djy23'
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('Incorrect password!')

Incorrect password!


- Case 3) OK

In [23]:
db_pwd

'mxsnWYLpB/nAOt37fpt3PMXpGGQprktT3vm2p9RfFyY='

In [22]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)       # gen_pwd()의 인수는 기본적으로 uid+'21'로 되어있다 
if db_pwd == hashed_pwd:        # 이것이 gen_pwd에서 연산되어 비밀번호가 만들어진다.
    print('Login')
else:
    print('Incorrect password!')

Login
